<a href="https://colab.research.google.com/github/PachecoLeonardo700/TECNOVAX/blob/main/APITESTfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar las librerías necesarias, la librería pillow la usamos para convertir una imagen a formato 64 para incluir en la creación de los trámites

In [ ]:
!pip install requests --quiet
print("requests instalado")

# 1) Instalar Pillow si no está instalado
!pip install pillow --quiet
print("Pillow instalado")

# 2) Subir y procesar PNG (preserva alpha/transparencia)
from google.colab import files
from PIL import Image
import io, base64, os

# Parámetros
MAX_WIDTH = 1024          # ancho máximo; ajusta si quieres menos (p. ej. 800)
OUTPUT_FORMAT = "PNG"     # mantiene PNG y transparencia
PNG_OPTIMIZE = True       # optimizar salida PNG
PNG_COMPRESSION = 6       # 0-9 (mayor = más compresión lento). Ajusta si hay límites.

print("Selecciona la imagen PNG desde tu equipo (aparecerá un selector).")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No se subió ninguna imagen. Cancelo.")

filename = next(iter(uploaded))
print("Archivo subido:", filename)

# Cargar imagen
img = Image.open(io.BytesIO(uploaded[filename]))
print("Modo original:", img.mode, "Tamaño original:", img.size)

# Si está muy ancha, redimensionar manteniendo proporción
w, h = img.size
if w > MAX_WIDTH:
    new_h = int((MAX_WIDTH / w) * h)
    img = img.resize((MAX_WIDTH, new_h), Image.LANCZOS)
    print(f"Redimensionada a: {img.size}")
else:
    print("No redimensionada.")

# Guardar como PNG en buffer (preservando alpha si existe)
buf = io.BytesIO()
save_kwargs = {}
if OUTPUT_FORMAT.upper() == "PNG":
    save_kwargs["optimize"] = PNG_OPTIMIZE
    # Pillow usa 'compress_level' 0-9
    save_kwargs["compress_level"] = PNG_COMPRESSION
img.save(buf, format="PNG", **save_kwargs)
buf.seek(0)
img_bytes = buf.read()

# Convertir a base64 (string sin prefijo)
IMAGE_BASE64 = base64.b64encode(img_bytes).decode("utf-8")
print("IMAGE_BASE64 creada. Longitud (bytes base64):", len(IMAGE_BASE64))

# Guardar info útil en variables (opcional para debug)
IMAGE_FILENAME = filename
IMAGE_SIZE_BYTES = len(img_bytes)
print("Tamaño PNG en bytes (aprox):", IMAGE_SIZE_BYTES)


Pillow instalado
Selecciona la imagen PNG desde tu equipo (aparecerá un selector).


Saving tramite.png to tramite.png
Archivo subido: tramite.png
Modo original: RGBA Tamaño original: (200, 200)
No redimensionada.
IMAGE_BASE64 creada. Longitud (bytes base64): 3000
Tamaño PNG en bytes (aprox): 2248


# Guardamos la información de la base de datos del sitio web (url del sitio, nombre de la bd, id del usuario administrador, clave api de acceso admin al sitio web y la url de la api externa de bruno para obtener los trámites)

In [ ]:
import os

# EDITA ESTO
ODOO_URL = "https://tramitespropuesta7.odoo.com/jsonrpc"  # endpoint JSON-RPC
ODOO_DB = "tramitespropuesta7"                           # nombre DB
ODOO_UID = 2                                            # tu user ID
ODOO_API_KEY = "5ef5362118a1798f6192025dad409f84b9e5c7bc"                        # tu API Key
EXTERNAL_API = "https://modmejorareg.devel.telecom.lat/api/tramites/listar"

os.environ["ODOO_URL"] = ODOO_URL
os.environ["ODOO_DB"] = ODOO_DB
os.environ["ODOO_UID"] = str(ODOO_UID)
os.environ["ODOO_API_KEY"] = ODOO_API_KEY
os.environ["EXTERNAL_API"] = EXTERNAL_API

print("Configuración establecida.")


Configuración establecida.


# Definimos el método que sirve para cargar los elementos en la bd

In [ ]:
import requests, json
from html import escape

def odoo_jsonrpc(model, method, args=None, kwargs=None):
    payload = {
        "jsonrpc": "2.0",
        "method": "call",
        "params": {
            "service": "object",
            "method": "execute_kw",
            "args": [
                os.environ["ODOO_DB"],
                int(os.environ["ODOO_UID"]),
                os.environ["ODOO_API_KEY"],
                model,
                method,
                args or [],
                kwargs or {}
            ]
        },
        "id": 1
    }
    r = requests.post(os.environ["ODOO_URL"], json=payload, timeout=60)
    r.raise_for_status()
    resp = r.json()
    if "error" in resp:
        raise Exception(json.dumps(resp["error"], indent=2))
    return resp.get("result")

# prueba mínima (opcional)
# print(odoo_jsonrpc("res.partner","search_read", args=[[["id",">",0]]], kwargs={"limit":1}))


# Definimos el método para obtener la información de la api solicitando solo los datos que nos resultan útiles para utilizar en el sitio web

In [ ]:
r = requests.get(os.environ["EXTERNAL_API"], timeout=30)
r.raise_for_status()
payload = r.json()
tramites_raw = payload.get("data") or []
print(f"Trámites obtenidos: {len(tramites_raw)}")

def pick_fields(t):
    return {
        "id": t.get("id"),
        "cNombreLegal": t.get("cNombreLegal"),
        "cHomoclave": t.get("cHomoclave"),
        "cDescripcion": t.get("cDescripcion"),
        "cInformacionContacto": t.get("cInformacionContacto"),
        "enumTipoCosto": t.get("enumTipoCosto"),
        "cCosto": t.get("cCosto"),
        "unidad_administrativa": t.get("unidad_administrativa") or {},
    }

tramites = [pick_fields(t) for t in tramites_raw]

# mostrar ejemplo
import pprint
pp = pprint.PrettyPrinter(indent=2)
if tramites:
    print("Ejemplo (primer trámite filtrado):")
    pp.pprint(tramites[0])


Trámites obtenidos: 373
Ejemplo (primer trámite filtrado):
{ 'cCosto': 'Por cabeza de ganado Vacuno $452.56(4 UMA), Por cada cabeza de '
            'ganado PORCINO: peso de hasta 50 kg. $113.14 (1 UMA),peso de '
            '50.01 hasta 85.00 Kg. $169.71  (1.5 UMA) peso de entre 85.01 '
            'hasta 135 kg. $ 226.28 (2 UMA), peso de mas de 135 kg. $ 339.42 '
            '(3 UMA), por cabeza de ganado OVINO $ 113.14 (1 UMA), por cabeza '
            'de ganado CAPRINO $ 113.14 (1 UMA), por cabeza de ganado EQUINO '
            '$452.56 (4 UMA).El valor de un UMA es de $113.14',
  'cDescripcion': 'Servicio de sacrificio y faenado de ganado bovino, porcino, '
                  'caprino, ovino y equino',
  'cHomoclave': 'MID-ABM-001',
  'cInformacionContacto': 'Calle 132 No. 239 entre 59b y 59d, Fraccionamiento '
                          'Yucalpetén',
  'cNombreLegal': 'Sacrificio  y faenado de Ganado',
  'enumTipoCosto': 'fijo',
  'id': 1,
  'unidad_administrativa': { 'cNombre': '

# Definimos el método que sirve para construir el apartado de información de cada trámite

In [ ]:
def build_description_ecommerce(t):
    e = escape
    ua = t.get("unidad_administrativa") or {}
    cNombre = e(ua.get("cNombre") or "")
    cHomoclave = e(t.get("cHomoclave") or "")
    cDescripcion = e(t.get("cDescripcion") or "")
    cInformacionContacto = e(t.get("cInformacionContacto") or "")
    enumTipoCosto = e(t.get("enumTipoCosto") or "")
    cCosto = e(t.get("cCosto") or "")

    html = (
f'<p style="text-align: center;" data-oe-version="2.0"><strong>Homoclave :&nbsp;</strong>{cHomoclave}</p>\n\n'
f'<p style="text-align: center;"><strong>Dependencia: </strong>{cNombre}</p>\n\n'
'<div data-name="Acordeón" data-snippet="s_accordion" class="s_accordion">\n'
'        <div id="myCollapse" class=" accordion">\n'
'            <div class="accordion-item position-relative z-1" data-name="Accordion Item">\n'
'                <button type="button" class="accordion-header accordion-button justify-content-between gap-2 bg-transparent h6-fs fw-bold text-decoration-none text-reset transition-none collapsed" data-bs-toggle="collapse" aria-expanded="false" id="accordion-button266980_1" data-bs-target="#myCollapseTab266980_1" aria-controls="myCollapseTab266980_1"><span class="flex-grow-1"><p>Descripción del Trámite</p></span></button>\n'
'                <div class="accordion-collapse collapse" data-bs-parent="#myCollapse" role="region" id="myCollapseTab266980_1" aria-labelledby="accordion-button266980_1">\n'
'                    <div class="accordion-body">\n'
f'                        <p>{cDescripcion}</p>\n'
'                    </div>\n'
'                </div>\n'
'            </div>\n'
'            <div class="accordion-item position-relative z-1" data-name="Accordion Item">\n'
'                <button type="button" class="accordion-header accordion-button justify-content-between gap-2 bg-transparent h6-fs fw-bold text-decoration-none text-reset transition-none collapsed" data-bs-toggle="collapse" aria-expanded="false" id="accordion-button266980_2" data-bs-target="#myCollapseTab266980_2" aria-controls="myCollapseTab266980_2"><span class="flex-grow-1"><p>Información de Contacto&nbsp;</p></span></button>\n'
'                <div class="accordion-collapse collapse" data-bs-parent="#myCollapse" role="region" id="myCollapseTab266980_2" aria-labelledby="accordion-button266980_2">\n'
'                    <div class="accordion-body">\n'
f'                    {cInformacionContacto}</div>\n'
'                </div>\n'
'            </div>\n'
'            <div class="accordion-item position-relative z-1" data-name="Accordion Item">\n'
'                <button type="button" class="accordion-header accordion-button justify-content-between gap-2 bg-transparent h6-fs fw-bold text-decoration-none text-reset transition-none collapsed" data-bs-toggle="collapse" aria-expanded="false" id="accordion-button266980_3" data-bs-target="#myCollapseTab266980_3" aria-controls="myCollapseTab266980_3"><span class="flex-grow-1"><p>Tipo de costo</p></span></button>\n'
'                <div class="accordion-collapse collapse" data-bs-parent="#myCollapse" role="region" id="myCollapseTab266980_3" aria-labelledby="accordion-button266980_3">\n'
'                    <div class="accordion-body">\n'
f'                    {enumTipoCosto}</div>\n'
'                </div>\n'
'            </div>\n'
'            <div class="accordion-item position-relative z-1" data-name="Accordion Item">\n'
'                <button type="button" class="accordion-header accordion-button justify-content-between gap-2 bg-transparent h6-fs fw-bold text-decoration-none text-reset transition-none" data-bs-toggle="collapse" aria-expanded="true" id="accordion-button266980_4" data-bs-target="#myCollapseTab266980_4" aria-controls="myCollapseTab266980_4"><span class="flex-grow-1"><p>Costo</p></span></button>\n'
'                <div class="accordion-collapse collapse show" data-bs-parent="#myCollapse" role="region" id="myCollapseTab266980_4" aria-labelledby="accordion-button266980_4">\n'
'                    <div class="accordion-body">'
f'{cCosto}</div>\n'
'                </div>\n'
'            </div>\n'
'        </div>\n'
'    </div>'
    )
    return html

# ejemplo
if tramites:
    print(build_description_ecommerce(tramites[0])[:800], "...\n")


<p style="text-align: center;" data-oe-version="2.0"><strong>Homoclave :&nbsp;</strong>MID-ABM-001</p>

<p style="text-align: center;"><strong>Dependencia: </strong>Abastos de Mérida</p>

<div data-name="Acordeón" data-snippet="s_accordion" class="s_accordion">
        <div id="myCollapse" class=" accordion">
            <div class="accordion-item position-relative z-1" data-name="Accordion Item">
                <button type="button" class="accordion-header accordion-button justify-content-between gap-2 bg-transparent h6-fs fw-bold text-decoration-none text-reset transition-none collapsed" data-bs-toggle="collapse" aria-expanded="false" id="accordion-button266980_1" data-bs-target="#myCollapseTab266980_1" aria-controls="myCollapseTab266980_1"><span class="flex-grow-1"><p>Descripción del T ...



# Definimos un método que nos resultara útil para asignar una etiqueta con la dependencia del trámite, si la dependencia ya existe asignar esa etiqueta existente, si no crearla.

In [ ]:
def get_or_create_tag(tag_name):
    """
    Busca tag por nombre exacto en product.tag. Si no existe, la crea.
    Devuelve el id de tag (int).
    """
    if not tag_name:
        return None
    # buscar exacto
    res = odoo_jsonrpc("product.tag", "search_read", args=[[["name","=", tag_name]]], kwargs={"fields":["id"], "limit":1})
    if res:
        return res[0]["id"]
    # crear
    new_id = odoo_jsonrpc("product.tag", "create", args=[{"name": tag_name}])
    return new_id


# Construimos el método para insertar los trámites en el sitio web

In [ ]:
def upsert_tramite_as_product(tr):
    name = (tr.get("cNombreLegal") or f"Trámite {tr.get('id') or ''}").strip()
    default_code = (tr.get("cHomoclave") or (f"TRAM-{tr.get('id')}" if tr.get("id") else "")).strip()
    if not default_code:
        return {"ok": False, "reason": "no_default_code"}

    # construir html ecommerce
    description_ecommerce = build_description_ecommerce(tr)

    # tag por unidad_administrativa.cNombre
    ua = tr.get("unidad_administrativa") or {}
    tag_name = (ua.get("cNombre") or "").strip()
    tag_id = None
    if tag_name:
        try:
            tag_id = get_or_create_tag(tag_name)
        except Exception as e:
            # no detener por fallo de tag; solo reportar
            print(f"Warning: no se pudo get/create tag '{tag_name}': {e}")
            tag_id = None

    # valores simples SOLO (no relaciones complejas)
    vals = {
        "name": name,
        "default_code": default_code,
        "type": "service",
        "list_price": 0.0,
        "standard_price": 0.0,
        "is_published": True,
        "sale_ok": True,
        "purchase_ok": False,
        "taxes_id": [(6,0,[])],        # lista vacía de impuestos
        "categ_id": False,
        "description_ecommerce": description_ecommerce,
    }

    # 👇 Agregar imagen si la variable global IMAGE_BASE64 existe
    if 'IMAGE_BASE64' in globals() and IMAGE_BASE64:
        vals["image_1920"] = IMAGE_BASE64

    # product_tag_ids: si tag_id existe, usar (6,0,[tag_id])
    if tag_id:
        vals["product_tag_ids"] = [(6,0,[tag_id])]

    # buscar producto existente por default_code
    recs = odoo_jsonrpc("product.template", "search_read",
                        args=[[["default_code","=", default_code]]],
                        kwargs={"fields":["id"], "limit":1})
    if recs:
        pid = recs[0]["id"]
        # actualizar
        odoo_jsonrpc("product.template", "write", args=[[pid], vals])
        return {"ok": True, "action": "updated", "id": pid}
    else:
        pid = odoo_jsonrpc("product.template", "create", args=[vals])
        return {"ok": True, "action": "created", "id": pid}


# Prueba con un solo registro para ver como se generan los trámites

In [ ]:
if tramites:
    ejemplo = tramites[0]
    html_ecom = build_description_ecommerce(ejemplo)
    ua = ejemplo.get("unidad_administrativa") or {}
    tag_name_test = ua.get("cNombre") or ""

    vals_example = {
        "name": ejemplo.get("cNombreLegal"),
        "default_code": ejemplo.get("cHomoclave") or f"TRAM-{ejemplo.get('id')}",
        "type": "service",
        "list_price": 0.0,
        "standard_price": 0.0,
        "is_published": True,
        "sale_ok": True,
        "purchase_ok": False,
        "taxes_id": [(6,0,[])],
        "categ_id": False,
        "description_ecommerce": html_ecom,
        # product_tag_ids será resuelto al crear/actualizar (aquí sólo mostramos cómo quedaría)
    }

    # --- añadir imagen al dry-run si existe IMAGE_BASE64 en la sesión de Colab ---
    if 'IMAGE_BASE64' in globals() and IMAGE_BASE64:
        # en dry-run mostramos solo el prefijo/truncado para no saturar la salida
        vals_example["image_1920"] = IMAGE_BASE64[:200] + "...(truncado para dry-run)"
        # si prefieres ver el tamaño real en bytes base64:
        vals_example["_image_base64_len"] = len(IMAGE_BASE64)

    # --- si ya resolviste un tag de prueba, puedes mostrar cómo se vería product_tag_ids ---
    # por ejemplo si tu get_or_create_tag devolvió tag_id_test
    if 'tag_id_test' in globals() and tag_id_test:
        vals_example["product_tag_ids"] = [(6,0,[tag_id_test])]

    import pprint
    print("Dry-run payload ejemplo (no afecta Odoo):")
    pp = pprint.PrettyPrinter(indent=2, width=160)
    pp.pprint(vals_example)
    print("Tag candidate:", tag_name_test)


Dry-run payload ejemplo (no afecta Odoo):
{ '_image_base64_len': 3000,
  'categ_id': False,
  'default_code': 'MID-ABM-001',
  'description_ecommerce': '<p style="text-align: center;" data-oe-version="2.0"><strong>Homoclave :&nbsp;</strong>MID-ABM-001</p>\n'
                           '\n'
                           '<p style="text-align: center;"><strong>Dependencia: </strong>Abastos de Mérida</p>\n'
                           '\n'
                           '<div data-name="Acordeón" data-snippet="s_accordion" class="s_accordion">\n'
                           '        <div id="myCollapse" class=" accordion">\n'
                           '            <div class="accordion-item position-relative z-1" data-name="Accordion Item">\n'
                           '                <button type="button" class="accordion-header accordion-button justify-content-between gap-2 bg-transparent h6-fs '
                           'fw-bold text-decoration-none text-reset transition-none collapsed" da

# Insertamos los trámites para crear nuevos, a aplicar actualizaciones en los que ya existan en el sitio web. El rastreo se realiza por medio de la homoclave

In [ ]:
created = 0
updated = 0
failed = 0
results = []

for t in tramites:
    try:
        r = upsert_tramite_as_product(t)
        results.append(r)
        if r.get("ok"):
            if r["action"] == "created":
                created += 1
            else:
                updated += 1
        else:
            failed += 1
    except Exception as e:
        failed += 1
        print(f"Error procesando tramite {t.get('id')}: {e}")

print("Resumen final:")
print("Total trámites:", len(tramites))
print("Creados:", created)
print("Actualizados:", updated)
print("Fallidos:", failed)
# Mostrar primeros 10 resultados
import itertools, pprint
pprint.pprint(list(itertools.islice(results, 10)))


Resumen final:
Total trámites: 373
Creados: 0
Actualizados: 373
Fallidos: 0
[{'action': 'updated', 'id': 20, 'ok': True},
 {'action': 'updated', 'id': 21, 'ok': True},
 {'action': 'updated', 'id': 22, 'ok': True},
 {'action': 'updated', 'id': 23, 'ok': True},
 {'action': 'updated', 'id': 24, 'ok': True},
 {'action': 'updated', 'id': 25, 'ok': True},
 {'action': 'updated', 'id': 26, 'ok': True},
 {'action': 'updated', 'id': 14, 'ok': True},
 {'action': 'updated', 'id': 18, 'ok': True},
 {'action': 'updated', 'id': 27, 'ok': True}]
